In [ ]:
#DataFrames manipulation
import pandas as pd

#Economic data from FRED
from fredapi import Fred

#Libraries for the Plotting
import holoviews as hv
from holoviews import opts, dim
from holoviews.plotting.links import RangeToolLink
from bokeh.models import HoverTool

#Librarie to save the plots to html object
import panel as pn
import param

hv.extension('bokeh')

# Download the Economic Data

In [ ]:
#FRED API key - Created a Free Account
FRED_API_key = ""
fred = Fred(api_key=FRED_API_key)

In [ ]:
#Define the Reference Rates (Two timeseries due to change in 2008)
fed_ref_rate_metrics = ["DFEDTAR", "DFEDTARL"]

#Download the Data from FRED and Manipulate the dataframe
fed_ref_rate_df = fred.get_series(fed_ref_rate_metrics[0])
fed_ref_rate_df = fed_ref_rate_df.reset_index()

fed_ref_rate_df = fed_ref_rate_df.rename(columns={'index':'Date',fed_ref_rate_df.columns[1]:'Fed Reference Rate'})

for metric in fed_ref_rate_metrics[1:]:
    fed_ref_rate_df_next = fred.get_series(metric)
    fed_ref_rate_df_next = fed_ref_rate_df_next.reset_index()
    fed_ref_rate_df_next = fed_ref_rate_df_next.rename(columns={'index':'Date',fed_ref_rate_df_next.columns[1]:'Fed Reference Rate'})
    fed_ref_rate_df = fed_ref_rate_df.merge(fed_ref_rate_df_next, how='outer')

In [ ]:
#Define the CPI Metric
cpi_metric = 'CPIAUCSL'

#Download the Data from FRED and Manipulate the dataframe
cpi_df = fred.get_series(cpi_metric)
cpi_df = cpi_df.reset_index()

cpi_df = cpi_df.rename(columns={'index':'Date',cpi_df.columns[1]:'CPI'})

#Calculate the CPI as a YoY change based on Index values
cpi_df['CPI'] = (cpi_df['CPI'].pct_change(periods=12)*100).round(2)

#Convert the dataframe from monthly to daily values
cpi_df = cpi_df.set_index('Date').resample('D').ffill()


In [ ]:
#Join the Fed Funds Rate and CPI Dataframes in 1 single Dataframe
eco_df = fed_ref_rate_df.merge(cpi_df, on='Date', how='left')
eco_df = eco_df.set_index('Date')
eco_df = eco_df.fillna(method='ffill')

#Save Data to CSV file
eco_df.to_csv('CPI and Fed Reference Rate.csv')

In [ ]:
#Join the Fed Funds Rate and CPI Dataframes in 1 single Dataframe
eco_df = fed_ref_rate_df.merge(cpi_df, on='Date', how='left')
eco_df = eco_df.set_index('Date')
eco_df = eco_df.fillna(method='ffill')

#Create One More Column with the Real Interest Rate
eco_df['Real Interest Rate'] = eco_df['Fed Reference Rate'] - eco_df['CPI']

#Rename Columns so that they are Printed in the Graph
eco_df = eco_df.rename(columns={'Fed Reference Rate':'Fed_Reference_Rate', 'Real Interest Rate':'Real_Interest_Rate'})

In [ ]:
#Analyze the file and define the Hike Cycles - Manual Job
#Instantiate the Object with the New File
eco_df_hike_df = pd.read_csv('CPI and Fed Reference Rate_with hike cycles.csv')

#Limit the Dataframe for Periods with Rate Hikes
eco_df_hike_df = eco_df_hike_df.dropna(subset=['Hike Cycle'])

In [ ]:
#Get a List of all Hike Cycles
hike_cycles = eco_df_hike_df['Hike Cycle'].unique()

In [ ]:
#Create a Dataframe with Only the Fed Reference Rate
fed_ref_rate_hike_df = eco_df_hike_df.drop(columns = 'CPI')

#Create the Final Dataframe with columns as the Hike Cycle
fed_ref_rate_hike_df_final = fed_ref_rate_hike_df[fed_ref_rate_hike_df['Hike Cycle']==hike_cycles[0]]
fed_ref_rate_hike_df_final['Fed Reference Rate'] = fed_ref_rate_hike_df_final['Fed Reference Rate'] - fed_ref_rate_hike_df_final['Fed Reference Rate'].iloc[0]
fed_ref_rate_hike_df_final = fed_ref_rate_hike_df_final.rename(columns={'Fed Reference Rate': fed_ref_rate_hike_df_final['Hike Cycle'].iloc[0]})
fed_ref_rate_hike_df_final = fed_ref_rate_hike_df_final.drop(columns=['Date','Hike Cycle'])
fed_ref_rate_hike_df_final = fed_ref_rate_hike_df_final.reset_index(drop=True)
fed_ref_rate_hike_df_final = fed_ref_rate_hike_df_final.reset_index().rename(columns= {'index':'Days'})

#Loop through the Hike Cycles to create a Dataframe where each column corresponds to a Hike Cycle
for i in range(1,len(hike_cycles)):
    fed_ref_rate_hike_df_final_next = fed_ref_rate_hike_df[fed_ref_rate_hike_df['Hike Cycle']==hike_cycles[i]]
    fed_ref_rate_hike_df_final_next['Fed Reference Rate'] = fed_ref_rate_hike_df_final_next['Fed Reference Rate'] - fed_ref_rate_hike_df_final_next['Fed Reference Rate'].iloc[0]
    fed_ref_rate_hike_df_final_next = fed_ref_rate_hike_df_final_next.rename(columns={'Fed Reference Rate': fed_ref_rate_hike_df_final_next['Hike Cycle'].iloc[0]})
    fed_ref_rate_hike_df_final_next = fed_ref_rate_hike_df_final_next.drop(columns=['Date','Hike Cycle'])
    fed_ref_rate_hike_df_final_next = fed_ref_rate_hike_df_final_next.reset_index(drop=True)
    fed_ref_rate_hike_df_final_next = fed_ref_rate_hike_df_final_next.reset_index().rename(columns= {'index':'Days'})
    fed_ref_rate_hike_df_final = fed_ref_rate_hike_df_final.merge(fed_ref_rate_hike_df_final_next, on='Days', how='outer')

#Set the Index to Days
fed_ref_rate_hike_df_final = fed_ref_rate_hike_df_final.set_index('Days')

In [ ]:
#Create a Dataframe with Only the CPI
cpi_hike_df = eco_df_hike_df.drop(columns = 'Fed Reference Rate')
cpi_hike_df['CPI'] = pd.to_numeric(cpi_hike_df['CPI'],errors='coerce')

#Create the Final Dataframe with columns as the Hike Cycle
cpi_hike_df_final = cpi_hike_df[cpi_hike_df['Hike Cycle']==hike_cycles[0]]
cpi_hike_df_final['CPI'] = cpi_hike_df_final['CPI'] - cpi_hike_df_final['CPI'].iloc[0]
cpi_hike_df_final = cpi_hike_df_final.rename(columns={'CPI': cpi_hike_df_final['Hike Cycle'].iloc[0]})
cpi_hike_df_final = cpi_hike_df_final.drop(columns=['Date','Hike Cycle'])
cpi_hike_df_final = cpi_hike_df_final.reset_index(drop=True)
cpi_hike_df_final = cpi_hike_df_final.reset_index().rename(columns= {'index':'Days'})

#Loop through the Hike Cycles to create a Dataframe where each column corresponds to a Hike Cycle
for i in range(1,len(hike_cycles)):
    cpi_hike_df_final_next = cpi_hike_df[cpi_hike_df['Hike Cycle']==hike_cycles[i]]
    cpi_hike_df_final_next['CPI'] = cpi_hike_df_final_next['CPI'] - cpi_hike_df_final_next['CPI'].iloc[0]
    cpi_hike_df_final_next = cpi_hike_df_final_next.rename(columns={'CPI': cpi_hike_df_final_next['Hike Cycle'].iloc[0]})
    cpi_hike_df_final_next = cpi_hike_df_final_next.drop(columns=['Date','Hike Cycle'])
    cpi_hike_df_final_next = cpi_hike_df_final_next.reset_index(drop=True)
    cpi_hike_df_final_next = cpi_hike_df_final_next.reset_index().rename(columns= {'index':'Days'})
    cpi_hike_df_final = cpi_hike_df_final.merge(cpi_hike_df_final_next, on='Days', how='outer')

#Set the Index to Days
cpi_hike_df_final = cpi_hike_df_final.set_index('Days')

In [ ]:
#Instantiate the Dataframe for the "Fed Reference Rate, US CPI, US Real Interest Rate" Graph
graph_df = eco_df

#Generate all curves
def getCurves(n):
    for column in graph_df.columns:
        hover = HoverTool(tooltips=[("Date", "@Date{%F}"), (column, f"@{column}")], formatters={'@Date': 'datetime'})  
        curve = hv.Curve(graph_df[column], label = column).opts(opts.Curve(tools=[hover]))
        curve = curve.opts(xticks=10)
        yield curve
        
source_curves, target_curves  = [], []
for curve in getCurves(2):
    
    src = curve.relabel('').opts(width=800, height=100, yaxis=None) 
    tgt = curve.opts(width=800, ylabel = 'Rate (%)')
    source_curves.append(src)
    target_curves.append(tgt)
    
# Link RangeTool for the first curves in the list.
RangeToolLink(source_curves[0],target_curves[0], axes=['x','y'])  

#Overlay the source and target curves 
overlaid_plot_src = hv.Overlay(source_curves).relabel('')
overlaid_plot_tgt = hv.Overlay(target_curves)

#Add an Horizontal Line at 0
hline = hv.HLine(0).opts(color='black', line_width=1, line_dash='5 5')
overlaid_plot_tgt = overlaid_plot_tgt * hline
overlaid_plot_tgt = overlaid_plot_tgt.relabel('Fed Reference Rate, US CPI, US Real Interest Rate').opts(
    height=400, legend_position='top')

# Layout the plot
full_graph_initial = (overlaid_plot_tgt + overlaid_plot_src).cols(1)
full_graph_initial = full_graph_initial.opts(merge_tools=False,shared_axes=False)

In [ ]:
#Instantiate the Dataframe for the "Speed of Fed Reference Rate Hike" Graph
graph_df = fed_ref_rate_hike_df_final

#Generate all curves
def getCurves(n):
    for column in graph_df.columns:
        hover = HoverTool(tooltips=[("Days", "@Days"), ('Change in Reference Rate (%)', f"@{column}")])  
        curve = hv.Curve(graph_df[column], label = column).opts(opts.Curve(tools=[hover]))
        yield curve
        
source_curves, target_curves  = [], []
for curve in getCurves(2):
    
    src = curve.relabel('').opts(width=800, height=100, yaxis=None) 
    tgt = curve.opts(width=800, ylabel = 'Change in Reference Rate (%)', xlabel = 'Number of Days Since First Hike')
    source_curves.append(src)
    target_curves.append(tgt)
    
#Get the graph with the target curves
overlaid_plot_tgt = hv.Overlay(target_curves).relabel('Speed of Fed Reference Rate Hike').opts(
    height=400, legend_position='top')

# Layout the plot
full_graph_ref_rate = overlaid_plot_tgt

In [ ]:
#Instantiate the Dataframe for the "Change in CPI following Reference Rate Hike" Graph
graph_df = cpi_hike_df_final

#Generate all curves
def getCurves(n):
    for column in graph_df.columns:
        hover = HoverTool(tooltips=[("Days", "@Days"), (column, f"@{column}")])  
        curve = hv.Curve(graph_df[column], label = column).opts(opts.Curve(tools=[hover]))
        yield curve
        
source_curves, target_curves  = [], []
for curve in getCurves(2):
    
    src = curve.relabel('').opts(width=800, height=100, yaxis=None) 
    tgt = curve.opts(width=800, ylabel = 'Change in CPI (%)', xlabel = 'Number of Days Since First Hike')
    source_curves.append(src)
    target_curves.append(tgt)
    
#Get the graph with the target curves
overlaid_plot_tgt = hv.Overlay(target_curves).relabel('Change in US CPI following Fed Reference Rate Hike').opts(
    height=400, legend_position='top')

# Layout the plot
full_graph_cpi = overlaid_plot_tgt

In [ ]:
#Save the Plots
p = pn.panel(full_graph_initial)
p.save('Initial_Graph.html', embed = True)

p = pn.panel(full_graph_ref_rate)
p.save('Reference_Rate_and_Hike_Cycles.html', embed = True)

p = pn.panel(full_graph_cpi)
p.save('CPI_and_Hike_Cycles.html', embed = True)